In [3]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import Window
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType
import datetime as dt
from pyspark.sql import functions as F
from pyspark.sql import Window
import os
import re
import datetime as dt

# Config

In [46]:
# ==========================================
# 사용자 설정 (원하는 지역과 시군구를 입력하세요)
# ==========================================
REGION = "경기도"          # "ex.서울특별시"
SIGUNGU = "용인시 처인구"    # "ex.강남구"

# 베이스 경로
OUTPUT_BASE = "/opt/spark/data/output/silver_stage_2"
TOJI_LIST_PATH = f"/opt/spark/data/output/silver_stage_1/{REGION}_{SIGUNGU.replace(' ', '_')}_filtered_final_toji"
TOJI_OWNER_PATH = "/opt/spark/data/tojidaejang/parquet/ocr_result.parquet"

In [7]:
# Spark Session 설정
spark = SparkSession.builder \
    .appName(f'silver_stage_2_{REGION}_{SIGUNGU.replace(" ", "_")}') \
    .master("spark://spark-master:7077") \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.sql.shuffle.partitions", "400") \
    .getOrCreate()

# 필터링된 토지 리스트 로드

In [22]:
filtered_toji_df = spark.read.parquet(TOJI_LIST_PATH)

print("로드 row 수:", filtered_final_toji_df.count())
filtered_toji_df.printSchema()
filtered_toji_df.show(5, truncate=False)

로드 row 수: 5740
root
 |-- 법정동명: string (nullable = true)
 |-- 본번: string (nullable = true)
 |-- 고유번호: string (nullable = true)
 |-- 지번: string (nullable = true)
 |-- 소유권변동일자: string (nullable = true)
 |-- 토지면적: double (nullable = true)
 |-- 지목: string (nullable = true)
 |-- 부번: string (nullable = true)
 |-- 관리_건축물대장_PK: string (nullable = true)
 |-- 옥외자주식면적: string (nullable = true)
 |-- 대장_구분_코드: string (nullable = true)
 |-- 업체명: string (nullable = true)
 |-- 대표자: string (nullable = true)
 |-- 소재지_정제: string (nullable = true)

+----------------------------------+----+-------------------+------+--------------+--------+----+----+------------------+--------------+--------------+-----------------------------+------+------------------------------------+
|법정동명                          |본번|고유번호           |지번  |소유권변동일자|토지면적|지목|부번|관리_건축물대장_PK|옥외자주식면적|대장_구분_코드|업체명                       |대표자|소재지_정제                         |
+----------------------------------+----+-------------------+------+----

# 토지 그룹 지주 로드

In [21]:
toji_owner_df = spark.read.parquet(TOJI_OWNER_PATH)

print("로드 row 수:", toji_owner_df.count())
toji_owner_df.printSchema()
toji_owner_df.show(5, truncate=False)

로드 row 수: 107
root
 |-- 주소: string (nullable = true)
 |-- 본번: string (nullable = true)
 |-- 부번: string (nullable = true)
 |-- 지주: string (nullable = true)
 |-- 소유권변동일자: string (nullable = true)

+---------------------------+----+----+------+--------------+
|주소                       |본번|부번|지주  |소유권변동일자|
+---------------------------+----+----+------+--------------+
|경기도 용인시 처인구 고림동|505 |30  |임채원|2024-03-11    |
|경기도 용인시 처인구 고림동|540 |2   |박장배|2022-06-02    |
|경기도 용인시 처인구 고림동|781 |48  |이병하|2011-07-14    |
|경기도 용인시 처인구 고림동|772 |2   |홍명선|2022-04-15    |
|경기도 용인시 처인구 고림동|781 |27  |김은년|1990-02-15    |
+---------------------------+----+----+------+--------------+
only showing top 5 rows


# 지주를 알고 있는 토지 리스트만 필터링 (inner join)

In [29]:
t = filtered_toji_df.alias("t")
o = toji_owner_df.alias("o")

toji_with_owner_df = (
    t.join(
        o,
        (F.col("t.법정동명") == F.col("o.주소")) &
        (F.col("t.본번") == F.col("o.본번")) &
        (F.col("t.소유권변동일자") == F.col("o.소유권변동일자")),
        how="inner"
    )
    .select(
        "t.*",                     
        F.col("o.지주").alias("지주"),
    )
)

print(toji_with_owner_df.count())
toji_with_owner_df.show(5)

149
+---------------------------+----+-------------------+-----+--------------+--------+----+----+------------------+--------------+--------------+------------------------+------+--------------------------------+------+
|                   법정동명|본번|           고유번호| 지번|소유권변동일자|토지면적|지목|부번|관리_건축물대장_PK|옥외자주식면적|대장_구분_코드|                  업체명|대표자|                     소재지_정제|  지주|
+---------------------------+----+-------------------+-----+--------------+--------+----+----+------------------+--------------+--------------+------------------------+------+--------------------------------+------+
|경기도 용인시 처인구 고림동| 747|4146110600107470001|747-1|    2015-11-19|   145.0|  대|   1|    11161100275260|          11.5|             1|                  현이네| 박*현|경기도 용인시 처인구 고림로11...|박병현|
|경기도 용인시 처인구 고림동| 464|4146110600104640000|  464|    2020-03-23|  1300.0|  대|NULL|    11161100465531|            50|             1|slint coffee(슬린트커피)| 변*은| 경기도 용인시 처인구 임원로 120|장시철|
|경기도 용인시 처인구 고림동| 342|4146110600103420001|34

# 지주가 음식점 대표인 토지 그룹 필터링

In [34]:
def name_match(col_rep, col_owner):
    return (
        (F.substring(col_rep, 1, 1) == F.substring(col_owner, 1, 1)) &
        (F.substring(col_rep, -1, 1) == F.substring(col_owner, -1, 1))
    )

# ---------------------------------
# 1. (법정동명, 본번, 지주) 그룹에
# 대표자 == 지주 존재 여부 계산
# ---------------------------------
match_group_df = (
    toji_with_owner_df
    .groupBy("법정동명", "본번", "지주")
    .agg(
        F.max(
            F.when(
                name_match(F.col("대표자"), F.col("지주")),
                1
            ).otherwise(0)
        ).alias("대표자지주일치")
    )
    .filter(F.col("대표자지주일치") == 1)
    .select("법정동명", "본번", "지주")
)

# ---------------------------------
# 2. 해당 그룹만 남기기 (inner join)
# ---------------------------------
filtered_match_df = (
    toji_with_owner_df
    .join(
        match_group_df,
        on=["법정동명", "본번", "지주"],
        how="inner"
    )
)

# ---------------------------------
# 3. 결과 확인
# ---------------------------------
print("필터 후 row 수:", filtered_match_df.count())
filtered_match_df.show(5, truncate=False)

필터 후 row 수: 5
+---------------------------+----+------+-------------------+------+--------------+--------+----+----+----------------------+--------------+--------------+--------------+------+-------------------------------------+
|법정동명                   |본번|지주  |고유번호           |지번  |소유권변동일자|토지면적|지목|부번|관리_건축물대장_PK    |옥외자주식면적|대장_구분_코드|업체명        |대표자|소재지_정제                          |
+---------------------------+----+------+-------------------+------+--------------+--------+----+----+----------------------+--------------+--------------+--------------+------+-------------------------------------+
|경기도 용인시 처인구 고림동|599 |박용성|4146110600105990017|599-17|2018-12-07    |74.0    |도로|17  |NULL                  |NULL          |NULL          |NULL          |NULL  |NULL                                 |
|경기도 용인시 처인구 고림동|599 |박용성|4146110600105990018|599-18|2018-12-07    |511.0   |대  |18  |11161100456536        |46            |1             |큰집          |박*성 |경기도 용인시 처인구 고림로200번길 5 |
|경기도 용인시 처인구 고림동|

# 주차장 면적 SUM

In [38]:
df = filtered_match_df.withColumn(
    "옥외자주식면적_d",
    F.regexp_replace(F.col("옥외자주식면적"), ",", "").cast("double")
)

df = df.withColumn(
    "주차장면적_기여",
    F.when(
        F.col("업체명").isNotNull(),
        F.coalesce(F.col("옥외자주식면적_d"), F.lit(0.0))
    ).when(
        (F.col("업체명").isNull()) & (F.col("지목").isin("차", "잡", "대")),
        F.coalesce(F.col("토지면적"), F.lit(0.0))
    ).otherwise(F.lit(0.0))
)

parking_by_group_df = (
    df.groupBy("법정동명", "본번", "지주")
      .agg(F.sum("주차장면적_기여").alias("주차장면적"))
)

parking_by_group_df.show(5, truncate=False)


+---------------------------+----+------+----------+
|법정동명                   |본번|지주  |주차장면적|
+---------------------------+----+------+----------+
|경기도 용인시 처인구 고림동|599 |박용성|46.0      |
|경기도 용인시 처인구 고림동|208 |최경배|12.5      |
|경기도 용인시 처인구 고림동|747 |박병현|11.5      |
+---------------------------+----+------+----------+



In [44]:
# 1) filtered_match_df에서 음식점(업체명 not null)만
restaurant_only_df = (
    filtered_match_df
    .filter(F.col("업체명").isNotNull())
    .select("고유번호","법정동명","본번","부번","관리_건축물대장_PK","업체명","대표자","지주")
)

# 2) 주차장면적 테이블과 join (키: 법정동명, 본번, 지주)
result_df = (
    restaurant_only_df.alias("r")
    .join(
        parking_by_group_df.alias("p"),
        on=["법정동명", "본번", "지주"],
        how="left"   # 음식점은 남기고 주차장면적 붙이기
    )
)

result_df.show(5, truncate=False)


+---------------------------+----+------+-------------------+----+----------------------+--------------+------+----------+
|법정동명                   |본번|지주  |고유번호           |부번|관리_건축물대장_PK    |업체명        |대표자|주차장면적|
+---------------------------+----+------+-------------------+----+----------------------+--------------+------+----------+
|경기도 용인시 처인구 고림동|208 |최경배|4146110600102080011|11  |1000000000000001619821|팔팔토종순대국|최*배 |12.5      |
|경기도 용인시 처인구 고림동|599 |박용성|4146110600105990018|18  |11161100456536        |큰집          |박*성 |46.0      |
|경기도 용인시 처인구 고림동|747 |박병현|4146110600107470001|1   |11161100275260        |현이네        |박*현 |11.5      |
+---------------------------+----+------+-------------------+----+----------------------+--------------+------+----------+



In [47]:
# =========================
# 저장 경로 생성
# =========================
result_path = os.path.join(
    OUTPUT_BASE,
    f"{REGION}_{SIGUNGU.replace(' ', '_')}_restaurant_parking"
)

print("저장 경로:", result_path)

# =========================
# parquet 저장
# =========================
result_df.write \
    .mode("overwrite") \
    .parquet(result_path)

print("✅ 저장 완료")


저장 경로: /opt/spark/data/output/silver_stage_2/경기도_용인시_처인구_restaurant_parking
✅ 저장 완료


In [48]:
spark.stop()